In [1]:
import os
import sys
import glob
import argparse
import matplotlib.pyplot as plt
os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
!KERAS_BACKEND=tensorflow python3 -c "from keras import backend"

In [2]:
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172 # unfreeze the top 2 inception blocks

def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    
    for layer in model.layers:
        print(layer.name, ': ', layer.trainable)
    model.compile(optimizer=RMSprop(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
      Args:
        base_model: keras model excluding top
        nb_classes: # of classes
      Returns:
        new keras model with last layer
    """
    
    x = base_model.output

    # GlobalAveragePooling2D converts the MxNxC tensor output into a 1xC tensor where C is the # of channels.
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) # new FC layer, random init
    # softmax function on the output to squeeze the values between [0,1]
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


In [4]:
def setup_to_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
  note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
  Args:
    model: keras model
  """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
def train():
    """Use transfer learning and fine-tuning to train a network on a new dataset"""
    
#     nb_train_samples = 45000
#     nb_classes = 15
#     nb_val_samples = 15000
#     nb_epoch = 15
#     batch_size = 4 #16
#     train_dir = 'data-all/train'
#     val_dir = 'data-all/validation'
#     output_model_file = 'inceptionv3_clothing_expanded_classifier.h5'

    nb_train_samples = 40000
    nb_classes = 10
    nb_val_samples = 10000
    nb_epoch = 15
    batch_size = 4 #16
    train_dir = 'data/train'
    val_dir = 'data/test'
    output_model_file = 'inceptionv3_clothing_classifier_v1.h5'


    # data prep
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input, # zero-centers our image data
        rescale = 1./255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    
    val_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input
    )
    
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )
    
    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=batch_size,
        class_mode='categorical'
    )
    
   

    # setup model
    # leave out the weights of the last fully connected layer
    base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, nb_classes)
    
    
    # transfer learning
    setup_to_transfer_learn(model, base_model)

    history_tl = model.fit_generator(
        train_generator,
        epochs=nb_epoch,
        steps_per_epoch=nb_train_samples//batch_size,
        validation_data=validation_generator,
        validation_steps=nb_val_samples//batch_size
    )
    
    print('Begin Fine Tuning')
    # fine-tuning
    setup_to_finetune(model)
    
    history_ft = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_val_samples//batch_size
    )
    
    model_json = model.to_json()
    with open("incep_filter_clothing_classifier_v1.json", 'w') as json_file:
        json_file.write(model_json)
    
    model.save(output_model_file)


In [6]:
train()

Found 43932 images belonging to 10 classes.
Found 10022 images belonging to 10 classes.
input_1 :  False
conv2d_1 :  False
batch_normalization_1 :  False
activation_1 :  False
conv2d_2 :  False
batch_normalization_2 :  False
activation_2 :  False
conv2d_3 :  False
batch_normalization_3 :  False
activation_3 :  False
max_pooling2d_1 :  False
conv2d_4 :  False
batch_normalization_4 :  False
activation_4 :  False
conv2d_5 :  False
batch_normalization_5 :  False
activation_5 :  False
max_pooling2d_2 :  False
conv2d_9 :  False
batch_normalization_9 :  False
activation_9 :  False
conv2d_7 :  False
conv2d_10 :  False
batch_normalization_7 :  False
batch_normalization_10 :  False
activation_7 :  False
activation_10 :  False
average_pooling2d_1 :  False
conv2d_6 :  False
conv2d_8 :  False
conv2d_11 :  False
conv2d_12 :  False
batch_normalization_6 :  False
batch_normalization_8 :  False
batch_normalization_11 :  False
batch_normalization_12 :  False
activation_6 :  False
activation_8 :  False
a